In [2]:
from Ks import *
import pickle as pkl

# load data
f = open("data.pickle", "rb")
data = pkl.load(f)

In [3]:
import numpy as np

features = []
labels = []

for i in data:
    tmp = [i.bigCategory, i.day, i.goal, i.backers]
    features.append(tmp)
    labels.append(i.pleged)

X = np.array(features)
y = np.array(labels)

In [4]:
from sklearn.preprocessing import OneHotEncoder, normalize

# one hot encode category
category = X[:, [0]]
enc = OneHotEncoder(handle_unknown='ignore')
category = enc.fit_transform(category).toarray()

In [5]:
# max normalize numeric features
d = X[:, [1,2,3]]
d = normalize(d, axis=0, norm="max")

In [6]:
X = np.concatenate((category, d), axis=1)

In [7]:
from sklearn.model_selection import KFold, train_test_split

# K-fold cross validation
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(X)

5

In [8]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

best_acc = 0

# use odd K to avoid tie
for i in range(11, 20, 2):

    for j, [train_index, test_index] in enumerate(kf.split(X)):
        X_train = X[train_index, :]
        y_train = y[train_index]

        X_test = X[test_index, :]
        y_test = y[test_index]

        # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print("Building on K={} fold={}".format(i, j))
        neigh = NearestNeighbors(n_neighbors=i)
        neigh.fit(X_train)

        print("Testing...")
        n = neigh.kneighbors(X_test, i, return_distance=False)
        pledge_predict = []
        state_predict = []
        actual_state = []

        for inx, nbrs in enumerate(n):
            nbrPlege = y_train[nbrs]
            p = np.mean(nbrPlege)
            pledge_predict.append(p)
            
            goal = X_test[inx][2]
            state = X_test[inx][2] >= y_test[inx]
            actual_state.append(state)

            # goal = X_test[inx][2]
            # state = X_test[inx][2] >= y_test[inx]
            # actual_state.append(state)

            if (p >= goal and state) or (p < goal and not state):
                state_predict.append(True)
            else:
                state_predict.append(False)
        
        RMSE = np.sqrt((np.square(pledge_predict - y_test).mean(axis=0)))
        tn, fp, fn, tp = confusion_matrix(state_predict, actual_state).ravel()
        acc = accuracy_score(actual_state, state_predict)
        
        print("           Positive       Negative")
        print("Positive     {}             {}".format(tn, fp))
        print("Negative     {}             {}".format(fn, tp))

        print(classification_report(state_predict, actual_state))
        print("Accuracy: {}".format(acc))
        print("RMSE: {}".format(RMSE))

        if acc > best_acc:
            # save best classifier
            clf = open("KNN_clf.pkl", "wb")
            pkl.dump(neigh, clf)
            clf.close()
            break
    break

Building on K=11 fold=0
Testing...
           Positive       Negative
Positive     94884             255
Negative     0             14077
              precision    recall  f1-score   support

       False       1.00      1.00      1.00     95139
        True       0.98      1.00      0.99     14077

    accuracy                           1.00    109216
   macro avg       0.99      1.00      0.99    109216
weighted avg       1.00      1.00      1.00    109216

Accuracy: 0.9976651772634046
RMSE: 61332.5808016165
